In [16]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/low'
folder = 'sp_matrix_tag_pop'
file_name = 'normalized_to_rating_filter_track_5_user_50.pkl'

In [17]:
predictions = pd.read_pickle(os.path.join(dir_, folder, 'topN_pred_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [18]:
predictions[:5]

,uid,tid,rating
0,0,2,0.070515
1,0,14,0.022937
2,0,15,0.147609
3,0,20,0.038996
4,0,21,0.023123


In [19]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [20]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)
    print(len(groups[0]))

2
3
4
5
6
7
8


In [21]:
num_user = len(predictions['uid'].unique())
num_user

218

In [7]:
# Personal Recommendation
n=20
top_n_lists = []

for i in tqdm(range(num_user)):
    prediction = predictions[predictions['uid'] == i]
    prediction = prediction.sort_values(by=['rating'],  ascending=False)
    prediction = prediction[:n]
#     print(prediction)
    top_n_list = []
    for _, row in prediction.iterrows():
        top_n_list.append(row[1])
#     print(top_n_list)
    top_n_lists.append(top_n_list)

In [8]:
evaluation = []
satisfication = 0
for i in tqdm(range(num_user)):
    top_n_list = top_n_lists[i]
    precision = 0
    for j in top_n_list:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision += 1
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/num_user)


0.16192660550458718


In [7]:
groups = np.loadtxt(os.path.join(dir_, 'groups', 'low' ,'2.csv'), delimiter=',')
group_size = len(groups[0])
groups[0]

array([116.,  84.])

In [8]:
n = 50
top_n_lists_g = []
    
for group in tqdm(groups):
    u1 = predictions[predictions['uid'] == group[0]]
    u2 = predictions[predictions['uid'] == group[1]]

    m = pd.merge(u1, u2, on=['tid'], how='inner', suffixes=['_u1','_u2'])
    m = m[['tid', 'rating_u1','rating_u2']]
    
    m['rating'] = (m['rating_u1'] + m['rating_u2'])/2
    m = m[['tid', 'rating']]
    m = m.sort_values(by=['rating'],  ascending=False)
    m = m[:n]
    top_n_list = []
    for _, row in m.iterrows():
        top_n_list.append(row[0])
    top_n_lists_g.append(top_n_list)
    
satisfication = 0
for i in tqdm(range(len(groups))):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    precision = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]

        s = len(t1) + len(t2)
        if s > 0:
            rating = 0.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
            if high_rating > 0:
                precision += 1
    satisfication += precision / n
print(satisfication/(len(groups)))


0.25899999999999995


In [13]:
precision_4 = []
precision_20 = []
precision_50 = []

recall_4 = []
recall_20 = []
recall_50 = []

In [14]:
# Precision and Recall
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_precisions = []
    avg_recalls = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        recalls =[]
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            
            for k in range(len(group)):
                high_rating = 0
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        
                precision = high_rating / top_n_size
                recall = high_rating / len(test[test['uid'] == uid])
                precisions.append(precision)
                recalls.append(recall)
                
        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
        
        avg_recall = 0
        for recall in recalls:
            avg_recall += recall
        avg_recall /= len(recalls)
        avg_recalls.append(avg_recall)
        
    if top_n_size == 4:
        precision_4.append(avg_precisions)
    elif top_n_size == 20:
        precision_20.append(avg_precisions)
    elif top_n_size == 50:
        precision_50.append(avg_precisions)
        
    if top_n_size == 4:
        recall_4.append(avg_recalls)
    elif top_n_size == 20:
        recall_20.append(avg_recalls)
    elif top_n_size == 50:
        recall_50.append(avg_recalls)


In [15]:
for i in precision_4:
    print(i)
print('------------------')

for i in precision_20:
    print(i)
print('------------------')

for i in precision_50:
    print(i)
print('------------------')

for i in recall_4:
    print(i)
print('------------------')

for i in recall_20:
    print(i)
print('------------------')

for i in recall_50:
    print(i)

[0.21125, 0.14444444444444443, 0.0985576923076923, 0.07666666666666666, 0.08333333333333333, 0.07142857142857142, 0.05625]
------------------
[0.16000000000000023, 0.10814814814814822, 0.07163461538461534, 0.050666666666666645, 0.05303030303030301, 0.04183673469387755, 0.034999999999999996]
------------------
[0.1303999999999997, 0.08385185185185175, 0.056538461538461475, 0.04426666666666669, 0.04333333333333335, 0.033877551020408174, 0.030500000000000006]
------------------
[0.002495378006187909, 0.002573747854434312, 0.0020630080320828015, 0.0017559247366439937, 0.002335732724285442, 0.002141359836281786, 0.002486414740555024]
------------------
[0.00879700811726297, 0.007952949122187473, 0.006513701927985425, 0.00623651181191809, 0.007962874925409593, 0.00835046954783877, 0.008945717918023496]
------------------
[0.01821761009207932, 0.01720008748403158, 0.014903375759244315, 0.015505669693960377, 0.01901242284093716, 0.015823025604027475, 0.017161796299156748]


In [22]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    for i in range(len(r)):
        r[i] = 2**r[i] -1
    
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, r_max, k, method=0):
    dcg_max = dcg_at_k(r_max, k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [23]:
nDCG_4 = []
nDCG_20 = []
nDCG_50 = []

In [24]:
# Precision and Recall
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_nDCGs = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        nDCGs = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            
            for k in range(len(group)):
                truth_rating = []
                uid = group[k]
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    
                    if len(t) > 0:
                        truth_rating.append(t.iloc[0]['rating'])
                    else:
                        truth_rating.append(0)
                max_rating = test[test['uid']==uid].sort_values(by=['rating'],  ascending=False)['rating'].values[:top_n_size]
                nDCG = ndcg_at_k(truth_rating, max_rating, top_n_size, method=1)
                nDCGs.append(nDCG)                
        avg_nDCG = 0
        for nDCG in nDCGs:
            avg_nDCG += nDCG
        avg_nDCG /= len(nDCGs)
        avg_nDCGs.append(avg_nDCG)
        
    if top_n_size == 4:
        nDCG_4.append(avg_nDCGs)
    elif top_n_size == 20:
        nDCG_20.append(avg_nDCGs)
    elif top_n_size == 50:
        nDCG_50.append(avg_nDCGs)

In [25]:
for i in nDCG_4:
    print(i)
print('------------------')

for i in nDCG_20:
    print(i)
print('------------------')

for i in nDCG_50:
    print(i)

[0.09474056642133913, 0.060037298278184444, 0.040695581374202476, 0.025121424485096567, 0.04096324313875455, 0.01831146879798749, 0.007455836729067378]
------------------
[0.07637569994325472, 0.05034327634868481, 0.03188882485611248, 0.02297180784236035, 0.028529871559874687, 0.016446611143091407, 0.011252801465022723]
------------------
[0.0678977991078245, 0.04682387283055496, 0.03226484361191761, 0.026499102641477548, 0.030741009218703427, 0.018044074994249825, 0.01453370981288112]
